In [1]:
import pandas as pd
import os # package for using operating system
from dotenv import load_dotenv
from snowflake.snowpark.session import Session # package for building and using Snowflake sessions
# from snowflake.cortex import Complete
from transformers import AutoTokenizer # package to select the fitting tokenizer for a pretrained model
from huggingface_hub import login # package for login and identifying to Huggingface
import pandas as pd # package for data manipulation
import json # package to work with .json
import math # package for mathematical operations

/home/lisa-linux/stuff/ma_automated_documentation/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


https://arxiv.org/pdf/2303.04048 --> reference-free & reference-based metric (direct-assessment or one-to-five stars)  
https://arxiv.org/pdf/2407.07959 --> llm evaluation


In [ ]:
with open('openstack_openstack.json', 'r') as f:
    loaded_data = json.load(f)

In [6]:
input_txt = loaded_data['readme']
repo_name = loaded_data['repo_owner']

In [4]:
input_txt

"OpenStack\n=========\n\nOpenStack is a collection of interoperable components that can be deployed\nto provide computing, networking and storage resources. Those infrastructure\nresources can then be accessed by end users through programmable APIs.\n\nThis repository just represents OpenStack as a collection of git submodules.\nYou can find the repositories for individual components at:\nhttps://opendev.org/openstack\n\nYou can learn more about the various components in OpenStack at:\nhttps://openstack.org/software\n\nTo learn more about how to contribute to OpenStack, please head to our\nContributor portal: https://www.openstack.org/community/\n\nTo learn more about how OpenStack is governed, you can visit:\nhttps://governance.openstack.org/\n\n\nWhy this repository ?\n---------------------\n\nOur continuous integration system, Zuul, gates all of the contained projects\nin an effective single timeline. This means that OpenStack, across all of the\nprojects, does already have a sequen

In [ ]:
prompt_evaluation = f'''
You are acting as a software development expert for the following GitHub repository "{repo_name}".
Your task is to evaluate the following README of the repository/project based on the provided questions:
    - q1: What is the goal of the project?
    - q2: Why is the project useful?
    - q3: How can users get started with the project?
    - q4: Where can users get help with your project?
    - q5: Who maintains and contributes to the project?
How well offers the provided README answers to each question? Score each question on a scale from 1 to 5, 
where 1 means "insufficient", and 5 "perfect". Provide your choosen score with a short explaination.
A good indication for a high score is: 1) there a section or paragraph where the answer of the questions is provided,
2) the content of the README clear structered and understandable 3) that a developer who reads the README understand the purpose of the project.
You're not allowed to skip a question or add any small talk!
Structure your response in the following format:
###
    "q1": [
        "score": "your_score",
        "explaination": "your_explaination"
        ],
    "q2": ...
###
README: {input_txt}
'''

In [ ]:
# alle noten definieren!
## oder von jeder "Note" eines ansehen.
## oder besonders schlechte und besonders gute readmes selbst ansehen (ich als experte ... haha) und (ein-zwei random aus der mitte
## oder große delta ansehen.(z.b. schlecht erzeugtes readme zu gutem originalem readme)
## plot von hecht kategorien auftragen (meines gegen das originale)

In [11]:
model = 'llama3.1-8b'
# specify llm parameters for summary creation
model_params = {
   'temperature': 0, # default: 0 https://docs.snowflake.com/en/sql-reference/functions/complete-snowflake-cortex --> Internetrecherche hat keine anderen Empfehlungen ergeben
   # 'top_p': # default: 0 https://docs.snowflake.com/en/sql-reference/functions/complete-snowflake-cortex
    'max_tokens': 400
}

In [12]:
# load .env file
load_dotenv(override=True)

# set up connection parameters for Snowflake connection
connection_params = {
    "account": os.environ['SNOWFLAKE_ACCOUNT'],
    "user": os.environ['SNOWFLAKE_USER'],
    "password": os.environ['SNOWFLAKE_USER_PASSWORD'],
    "role": 'SYSADMIN',
    "warehouse": 'COMPUTE_WH',
    'paramstyle': 'qmark'
}

# build Snowflake session with connection parameters
snowflake_session = Session.builder.configs(connection_params).create()
print('Snowflake sessions is build.')
print('---------------------------------------------')

Snowflake sessions is build.
---------------------------------------------


In [14]:
query = f"""
            SELECT SNOWFLAKE.CORTEX.COMPLETE( 
                ?,
                [
                    {{
                        'role': 'user', 
                        'content': ?
                    }}
                ],
                {{
                    'temperature': ?,
                    'max_tokens':  ?
                }} 
            ) AS response
        """
response = snowflake_session.sql(query, params=[model, prompt_evaluation, model_params['temperature'], model_params['max_tokens']]).collect()
res = json.loads(response[0]['RESPONSE']) # load response as json object and save it in 'res'
# split 'res' parts and save them into multiple variabels
message = res['choices'][0]['messages'] 

In [18]:
print(message)

Here are the scores for each question:

```
"q1": [
    "score": 4,
    "explaination": "The goal of the project is not explicitly stated in a clear and concise manner, but it can be inferred from the text. The project is a collection of interoperable components that provide computing, networking, and storage resources, and it's intended to be used in a read-only manner to capture the sequence of commits that have been tested together."
],
"q2": [
    "score": 5,
    "explaination": "The project is useful because it provides a way to capture the sequence of commits that have been tested together, which can be important for ensuring the stability and reliability of the system. This is clearly explained in the 'Why this repository?' section."
],
"q3": [
    "score": 4,
    "explaination": "The README provides instructions on how to get started with the project, but it's not very clear. It mentions that the repository is intended to be used in a read-only manner, but it doesn't explain ho

In [19]:
snowflake_session.close()